### 0. Helper functions

In [147]:
import numpy as np

class bin_data:
    def __init__(self, env):
        self.total_capacity = np.full(env.num_bins, 0) # Total capacity seen across all trials
        self.capacity_used = np.full(env.num_bins, 0) # Total capacity used across all trials
        self.steps_taken = 0 # Number of timesteps taken before trial terminated
        self.placed = 0 # Number of items correctly placed
        self.misplaced = 0
        self.discarded = 0
        self.data_points = 0 # Number of trials we've witnessed
    

    def log(self, env):
        self.total_capacity = np.add(self.total_capacity, env.capacity)
        self.capacity_used = np.add(self.capacity_used, env.state[:-1])
        self.steps_taken += sum(env.logs.values())
        self.placed += env.logs["placed"]
        self.misplaced += env.logs["misplaced"]
        self.discarded += env.logs["discarded"]
        self.data_points += 1
        

    def get_avg(self):
        percentages = 100 - ((env.state/env.capacity) * 100)
        
        return {
            "steps" : self.steps_taken / self.data_points,
            "utilization" : sum(percentages) / len(percentages),
            "accuracy" : (self.placed / (self.placed + self.misplaced)) * 100
        }
    

    def print_data(self):
        data = self.get_avg()
        print(f'Average number of steps taken: {data["steps"]}')
        print(f'Average bin utilization: {round(data["utilization"], 2)}%')
        print(f'Accuracy: {round(data["accuracy"], 2)}%')

### 1. Import dependencies

In [ ]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [3]:
from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [21]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [60]:
env.action_space.sample()

1

In [108]:
env.observation_space.sample()

array([16, 14, 19,  0, 15,  9,  3, 10,  2, 14,  7])

### 3. Run baseline test (No ML)

In [150]:
control_data = bin_data(env)

In [151]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    control_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-192
{'placed': 44, 'misplaced': 236, 'discarded': 26}
Episode:2 Score:-235
{'placed': 47, 'misplaced': 282, 'discarded': 37}
Episode:3 Score:-423
{'placed': 46, 'misplaced': 469, 'discarded': 65}
Episode:4 Score:-58
{'placed': 40, 'misplaced': 98, 'discarded': 12}
Episode:5 Score:-462
{'placed': 44, 'misplaced': 506, 'discarded': 69}
Episode:6 Score:-549
{'placed': 48, 'misplaced': 597, 'discarded': 54}
Episode:7 Score:-231
{'placed': 46, 'misplaced': 277, 'discarded': 31}
Episode:8 Score:-77
{'placed': 42, 'misplaced': 119, 'discarded': 12}
Episode:9 Score:-208
{'placed': 56, 'misplaced': 264, 'discarded': 44}
Episode:10 Score:-130
{'placed': 54, 'misplaced': 184, 'discarded': 27}


### 4. Train an RL Model

In [31]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [32]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [33]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [34]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [156]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_42
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 167      |
|    ep_rew_mean     | 25       |
| time/              |          |
|    fps             | 975      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 174        |
|    ep_rew_mean          | 26.9       |
| time/                   |            |
|    fps                  | 745        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01022795 |
|    clip_fraction        | 0.128      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.53      |
|    explained_variance   | 0.741      |
|    learning

### 5. Save model

In [36]:
#PPO_Path = os.path.join('Training', 'Saved Models', 'Constant_PPO_Model_Discard_Penalty')

In [37]:
#model.save(PPO_Path)

### 6. Load model

In [38]:
#model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [161]:
real_data = bin_data(env)

In [162]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    real_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:51
{'placed': 57, 'misplaced': 6, 'discarded': 67}
Episode:2 Score:56
{'placed': 60, 'misplaced': 4, 'discarded': 92}
Episode:3 Score:47
{'placed': 56, 'misplaced': 9, 'discarded': 129}
Episode:4 Score:51
{'placed': 64, 'misplaced': 13, 'discarded': 137}
Episode:5 Score:57
{'placed': 60, 'misplaced': 3, 'discarded': 82}
Episode:6 Score:56
{'placed': 58, 'misplaced': 2, 'discarded': 69}
Episode:7 Score:66
{'placed': 71, 'misplaced': 5, 'discarded': 166}
Episode:8 Score:53
{'placed': 58, 'misplaced': 5, 'discarded': 99}
Episode:9 Score:52
{'placed': 57, 'misplaced': 5, 'discarded': 92}
Episode:10 Score:56
{'placed': 68, 'misplaced': 12, 'discarded': 156}


In [163]:
control_data.print_data()

Average number of steps taken: 387.6
Average bin utilization: 98.64%
Accuracy: 13.35%


In [164]:
real_data.print_data()

Average number of steps taken: 176.2
Average bin utilization: 98.64%
Accuracy: 90.49%
